In [32]:
import pandas as pd
from tensorflow.keras.preprocessing import image

import tensorflow as tf

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet_v2 import ResNet152V2


from tensorflow.keras import layers
from tensorflow.keras import models
from tqdm import tqdm
import numpy as np

In [33]:
df = pd.read_csv('./data/HAM10000_Metadata')

Actinic keratoses and intraepithelial carcinoma / Bowen's disease (__akiec__), 0

basal cell carcinoma (__bcc__), 1

benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses, __bkl__), 2

dermatofibroma (__df__),  3

melanoma (__mel__),  4

melanocytic nevi (__nv__)   5

vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage, __vasc__). 6 


More than 50% of lesions are confirmed through histopathology (histo), the ground truth for the rest of the cases is either follow-up examination (follow_up), expert consensus (consensus), or confirmation by in-vivo confocal microscopy (confocal). The dataset includes lesions with multiple images, which can be tracked by the lesion_id-column within the HAM10000_metadata file.

In [34]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [35]:
# VGG-16
preprocessed = []

model = VGG16(weights='imagenet', include_top=False)

for file in tqdm(df['image_id']):
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg16.preprocess_input(img)
    preds = model.predict(img)
    preprocessed.append(preds)
    
df['VGG16'] = preprocessed

100%|██████████| 10015/10015 [2:33:44<00:00,  1.09it/s] 


In [36]:
# Inception
preprocessed = []

model = InceptionV3(weights='imagenet', include_top=False)

for file in tqdm(df['image_id']):
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (299,299))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    preds = model.predict(img)
    preprocessed.append(preds)
    
df['IV3'] = preprocessed

100%|██████████| 10015/10015 [1:03:17<00:00,  2.64it/s]


In [37]:
# ResNet
preprocessed = []

model = ResNet152V2(weights='imagenet', include_top=False)

for file in tqdm(df['image_id']):
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.resnet_v2.preprocess_input(img)
    preds = model.predict(img)
    preprocessed.append(preds)
    
df['resnet'] = preprocessed

100%|██████████| 10015/10015 [2:05:42<00:00,  1.33it/s] 


In [38]:
labels = []
for i in range(0, 10015):
    dx = df.iloc[i]['dx']
    if dx == 'akiec':
        labels.append(tf.one_hot(0, 7))
    elif dx == 'bcc':
        labels.append(tf.one_hot(1, 7))
    elif dx == 'bkl':
        labels.append(tf.one_hot(2, 7))
    elif dx == 'df':
        labels.append(tf.one_hot(3, 7))
    elif dx == 'mel':
        labels.append(tf.one_hot(4, 7))
    elif dx == 'nv':
        labels.append(tf.one_hot(5, 7))
    elif dx == 'vasc':
        labels.append(tf.one_hot(6, 7))
        
df['one_hot'] = labels

In [19]:
import ast
df = pd.read_csv('./data/data.csv')

In [39]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,VGG16,IV3,resnet,one_hot
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,[[[[ 0. 0. 11.025178 0. ...,[[[[-0. -0. 0.34455207 ... 0...,"[[[[-0. -0. -0. ... -0. -0. -0.], [-0. -0. -0....","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,[[[[ 0. 0. 1.2555903 0.724...,[[[[-0. -0. -0. ... 0...,"[[[[-0. -0. -0. ... -0. -0. -0.], [-0. -0. -0....","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,[[[[ 0. 0. 6.3895373 0. ...,[[[[-0. -0. -0. ... 0.76...,"[[[[-0. -0. -0. ... -0. -0. -0.], [-0. -0. -0....","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,[[[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ...,[[[[ 1.1053773 -0. -0. ... 0...,"[[[[-0. -0. -0. ... -0. -0. -0.], [-0. -0. -0....","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,[[[[ 0. 0. 0. 0. ...,[[[[-0. -0. 0.10026377 ... 0...,"[[[[-0. -0. -0. ... -0. -0. -0.], [-0. -0. -0....","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T..."


In [49]:
type(df['one_hot'][0])

tensorflow.python.framework.ops.EagerTensor

In [50]:
VGG_X = np.array(df['VGG16'])
IV3_X = np.array(df['IV3'])
RES_X = np.array(df['resnet'])

In [51]:
y = np.array(df['one_hot'])

In [52]:
VGG_X[0]

array([[[[ 0.       ,  0.       , 11.025178 , ...,  0.       ,
           1.7154353,  0.       ],
         [ 0.       ,  0.       ,  5.5877066, ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       , 13.220544 , ...,  0.       ,
           0.       ,  0.       ],
         ...,
         [ 0.       ,  0.       , 10.496432 , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  5.7669587, ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       , 10.018325 , ...,  0.       ,
           0.       ,  0.       ]],

        [[ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         ...,
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           3.4823925,  0.

In [62]:
IV3_X[0].shape

(1, 8, 8, 2048)

In [105]:
model = models.Sequential()
model.add(layers.Flatten(input_shape=(7,7,512)))
model.add(layers.Dense(8192, activation='relu'))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(7))

In [106]:
new_VGG = []
for x in VGG_X:
    new_VGG.append(x.reshape(7, 7, 512))

In [107]:
new_VGG = np.asarray(new_VGG).astype('float32')

In [108]:
new_VGG.shape

(10015, 7, 7, 512)

In [109]:
new_y = []
for ele in y:
    new_y.append(np.array(ele))
    
new_y = np.array(new_y)

new_y.shape

(10015, 7)

In [110]:
model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
model.fit(new_VGG, new_y, epochs=10, validation_split=0.1)

Train on 10015 samples
Epoch 1/10
   96/10015 [..............................] - ETA: 11:09 - loss: 102.7127 - accuracy: 0.6094

KeyboardInterrupt: 

In [97]:
new_RES = []
for x in RES_X:
    new_RES.append(x.reshape(7, 7, 2048))

In [113]:
new_RES = np.array(new_RES)

In [114]:
res_model = models.Sequential()
res_model.add(layers.Flatten(input_shape=(7,7,2048)))
res_model.add(layers.Dense(8192, activation='relu'))
res_model.add(layers.Dense(4096, activation='relu'))
res_model.add(layers.Dense(2048, activation='relu'))
res_model.add(layers.Dense(1024, activation='relu'))
res_model.add(layers.Dense(512, activation='relu'))
res_model.add(layers.Dense(128, activation='relu'))
res_model.add(layers.Dense(7))

In [117]:
res_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
res_model.fit(new_RES, new_y, epochs=10, validation_split=0.1)

Train on 10015 samples
Epoch 1/10
   32/10015 [..............................] - ETA: 1:34:13

KeyboardInterrupt: 

In [123]:
new_IV3 = []
for x in IV3_X:
    new_IV3.append(x.reshape(8, 8, 2048))
new_IV3 = np.array(new_IV3)

In [124]:
iv3_model = models.Sequential()
iv3_model.add(layers.Flatten(input_shape=(8,8,2048)))
iv3_model.add(layers.Dense(8192, activation='relu'))
iv3_model.add(layers.Dense(4096, activation='relu'))
iv3_model.add(layers.Dense(2048, activation='relu'))
iv3_model.add(layers.Dense(1024, activation='relu'))
iv3_model.add(layers.Dense(512, activation='relu'))
iv3_model.add(layers.Dense(128, activation='relu'))
iv3_model.add(layers.Dense(7))

In [ ]:
iv3_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
iv3_model.fit(new_IV3, new_y, epochs=10, validation_split=0.1)

Train on 9013 samples, validate on 1002 samples
Epoch 1/10
